In [ ]:
# pip install transformers torch sentence-transformers
# pip install PyPDF2
# pip install sentence-transformers
#!pip install openai==0.28
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import spacy
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
#import openai
#from transformers import LlamaTokenizer, LlamaForCausalLM
from sentence_transformers import SentenceTransformer, util

import PyPDF2
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
with open("/kaggle/input/bladerunner/blade runner 2049.pdf", "rb") as pdf_file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Initialize an empty string to store the extracted text
    text = ''

    # Loop through each page and extract the text
    for page in pdf_reader.pages:
        text += page.extract_text()

In [ ]:
num_parts = 1250
part_size = len(text) // num_parts
text_parts = []
for i in range(num_parts):
    start_index = i * part_size
    end_index = (i + 1) * part_size if i < num_parts - 1 else len(text)
    text_part = text[start_index:end_index]
    text_parts.append(text_part)


In [ ]:
#import spacy
def evaluate(text_parts,my_query):  
#     nlp = spacy.load("en_core_web_md")
#     # Define the query and convert it to a spaCy Doc object
#     query = my_query
#     query_doc = nlp(query)

#     # Calculate semantic similarity scores between the query and each sentence
#     scores = []
#     for text_part in text_parts:
#         sentence_doc = nlp(text_part)
#         similarity_score = query_doc.similarity(sentence_doc)
#         scores.append((text_part, similarity_score))

#     # Sort sentences by similarity score (optional)
#     scores.sort(key=lambda x: x[1], reverse=True)
    model = SentenceTransformer('all-MiniLM-L6-v2')
#     # Define the query
    query = my_query

    # Compute embedding for the query and the sentences
    query_embedding = model.encode([query], convert_to_tensor=True)
    sentence_embeddings = model.encode(text_parts, convert_to_tensor=True)

    # Compute cosine-similarities
    cosine_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)


    # Create a list of tuples containing sentence and cosine similarity score
    scores = [(text_parts[i], cosine_scores[0][i].item()) for i in range(len(text_parts))]

    # Sort scores in decreasing order
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Create a dictionary to store total weighted score for each sentence
    score_dict = {}
    weight = 1
    #for pairs, weight in zip([normalized_scores], [0.2, 0.6, 0.2]):
    for text_part, score in scores:
        if text_part not in score_dict:
            score_dict[text_part] = 0
        score_dict[text_part] += weight * score

    # Now score_dict contains the weighted average score for each sentence
    weighted_scores = list(score_dict.items())

    # Sort scores in decreasing order
    weighted_scores.sort(key=lambda x: x[1], reverse=True)


    return weighted_scores

In [ ]:
model_name = "google/flan-t5-large"  # You can choose larger models like flan-t5-large
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
question = "Who is the writer?"
#question =  "Explain the theme of the movie"
#question =  "Who are the characters?"
#question =  "Does the script pass the Bechdel test?"
weighted_scores = evaluate(text_parts, question)

# Construct the context by combining the top-weighted scores
context = ". ".join([score[0] for score in weighted_scores[:min(10,len(weighted_scores))]]) + "."

# Create a prompt with the context and the question
prompt = f"{context}\nQuestion: {question}\nAnswer:"

input_ids = tokenizer.encode(prompt, return_tensors='pt')
#output = model.generate(input_ids, max_length=50)

output = model.generate(
    input_ids,
    max_length=500,  # Increase max_length for longer responses
    temperature=1.1,  # Adjust temperature for more varied output
    top_k=50,  # Control sampling diversity
    top_p=0.95,  # Control the probability distribution for output tokens
    num_return_sequences=1,  # Return multiple sequences if you want to select the best one
    do_sample=True  # Enable sampling for varied outputs
)

# Decode the output to get the generated answer
answer = tokenizer.decode(output[0], skip_special_tokens=True)
#print(context)
print("Answer:", answer)